# Importação

In [66]:
import os
import glob
import pandas as pd

from typing import List, Dict, TextIO, DefaultDict, Tuple
from collections import defaultdict

import commands_dictionary

In [67]:
path_logs = r'/mnt/c/Users/mathe/Documents/AIML_logs/treinamento_aiml/logs'

In [68]:
logs = pd.DataFrame()

# Funções

In [69]:
def get_equipment_type(path: str) -> List[str]:
    """Returns a list with equipment's types by iterating in the path folder.
    
    ## Args:
        path (str): Logs path.
        
    ## Returns:
        List[str]: List with equipment's types.
    """
    
    equipment_types = []
    entries = os.listdir(path)
    for entry in entries:
        equipment_types.append(entry)
    return equipment_types

In [70]:
def get_run_ids(row, path: str) -> List[str]:
    """Returns a list containing the run ids names by iterating in the path folder.

    ## Args:
        path (str): Logs path.
        
    ## Returns:
        str: Run ids
    """
    
    entries = os.listdir(f'{path}/{row["equipment_type"]}')
    return entries
    # for entry in entries:
    #     if entry == '220818_171756_851_8721_win':
    #         return entry

In [71]:
def get_test_case(row, path: str) -> List[str]:
    """Returns a list containing the number of test cases by iterating over every file in the path folder.

    ## Args:
        path (str): Logs path.
        
    ## Returns:
        List[str]: Dict with run_ids as keys and a list of test_cases names as values.
    """
    
    test_cases = []
    try:
        files = next(os.walk(f'{path}/{row["equipment_type"]}/{row["run_id"]}'))[2]
        for file in files:
            if file.endswith('.txt', -4) and file != 'seed.txt': ##and file == 'test_case_6042.txt':
                test_cases.append(file[10:-4])
    except StopIteration:
        return test_cases
    return test_cases

In [72]:
def open_file(path: str, eq_type: str, run_id: str, test_case: str) -> TextIO:
    """Open a test_case_*.txt file by receiving the path of the equipments, equipment type, run_id 
    and the number of the test case as arguments. Returns the file.

    ## Args:
        path (str): path where each equipment is located.
        eq_type (str): equipment_type.
        run_id (str): run_id.
        test_case (str): number of the test_case file.
        
    ## Returns:
        TextIO: the opened test_case file.
    """
    
    file = open(f'{path}/{eq_type}/{run_id}/test_case_{test_case}.txt', 'r', encoding='cp860')
    return file

In [73]:
def get_timestamp(row, path: str) -> str:
    """Returns the timestamps of every TX command in the log file by iterating every line in the log
    and checking if there is a "OPTICALTX" or "TX" substring in the line.

    ## Args:
        row (pd.Series): the row of the pd.DataFrame
        path (str): Logs path

    ## Returns:
        str: timestamps of the row's TX command
    """

    timestamps = []
    file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
    log_lines = file.readlines()
    for line in log_lines:
        if 'OPTICALTX:' in line or 'TX:' in line:
            timestamps.append(line[1:24])
    file.close()
    return timestamps

In [74]:
def get_baud_rate(path: str) -> Dict[str, str]:
    baud_rate = defaultdict(str)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                log_lines = file.readlines()
                for line in log_lines:
                    if 'OPTICALTX:' in line or 'TX:' in line:
                        baud_rate[line[1:24]] = line[27:31]
                file.close()
    return baud_rate

def apply_baud_rate(row: pd.Series, baud_dict: Dict[str, str]) -> str:
    return baud_dict.get(row['timestamp'])

In [75]:
def get_slot(path: str) -> str:
    """Returns the slot of every TX command in the log file by iterating every line in the log
    and checking if there is a "OPTICALTX" or "TX" substring in the line and if the timestamps match.

    ## Args:
        row (pd.Series): the row of the pd.DataFrame.
        path (str): Logs path.

    ## Returns:
        str: slot of the row's TX command.
    """
    slots = defaultdict(str)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                log_lines = file.readlines()
                for line in log_lines:
                    if 'OPTICALTX:' in line or 'TX:' in line:
                        slots[line[1:24]] = line[35:38].strip()
                file.close()
    return slots

def apply_slot(row: pd.Series, slots_dict: Dict[str, str]) -> str:
    return slots_dict.get(row['timestamp'])

In [76]:
def get_serial_port(path: str) -> str:
    """Returns the serial port of every TX command in the log file by iterating every line in the log
    and checking if there is a "OPTICALTX" or "TX" substring in the line and if the timestamps match.

    ## Args:
        row (pd.Series): the row of the pd.DataFrame.
        path (str): Logs path.
    ## Returns:
        str: serial port of the row's TX command.
    """
    serial_ports = defaultdict(str)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                log_lines = file.readlines()
                for line in log_lines:
                    if 'OPTICALTX:' in line:
                        serial_ports[line[1:24]] = line[40:49]
                    elif 'TX:' in line:
                        serial_ports[line[1:24]] = line[40:45]
                file.close()
    return serial_ports

def apply_serial_port(row: pd.Series, serial_ports_dict: Dict[str, str]) -> str:
    return serial_ports_dict.get(row['timestamp'])

In [77]:
def get_tx_data(path: str) -> str:
    """Returns the TX data of every TX command in the log file by iterating every line in the log
    and checking if there is a "OPTICALTX" or "TX" substring in the line and if the timestamps match.

    ## Args:
        row (pd.Series): the row of the pd.DataFrame.
        path (str): Logs path.

    ## Returns:
        str: tx data of the row's tx command.
    """
    tx_datas = defaultdict(str)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                log_lines = file.readlines()
                for line in log_lines:
                    if 'OPTICALTX:' in line or 'TX:' in line:
                        tx_datas[line[1:24]] = line[51:].rstrip('\n')
                file.close()
    return tx_datas
    
def apply_tx_data(row: pd.Series, tx_dict: Dict[str, str]) -> str:
    return tx_dict.get(row['timestamp'])

In [78]:
def get_rx_data(path: str) -> str:
    """Returns the RX data of its respective TX command by iterating every line in the log and checking
    if "TX" or "OPTICALTX" is in line and if it equals to the "tx_data" of the row and if the timestamps 
    match, if it evaluates to true it searches for a "RX response" with the same "serial_port" of the "TX command".

    ## Args:
        row (pd.Series): the row of the pd.DataFrame.
        path (str): Logs path.

    ## Returns:
        str: rx data of the row's tx command.
    """
    rx_datas = defaultdict(str)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                log_lines = file.readlines()
                for i, line in enumerate(log_lines):
                    if 'OPTICALTX:' in line or 'TX:' in line:
                        try:
                            j = 1
                            while True:
                                rx_line = log_lines[i+j]
                                if 'OPTICALRX:' in rx_line:
                                    rx_datas[(line[1:24], line[40:49])] = rx_line[51:].rstrip('\n')
                                    break
                                elif 'OPTICALTX:' in rx_line:
                                    j += 1
                                    continue
                                elif 'RX:' in rx_line:
                                    rx_datas[(line[1:24], line[40:45])] = rx_line[51:].rstrip('\n')
                                    break
                                elif 'TX:' in rx_line:
                                    j += 1
                                    continue
                                else:
                                    print('não encontrado', test_case)
                        except IndexError:
                            pass
                file.close()
    return rx_datas

def apply_rx_data(row: pd.Series, rx_dict: Dict[Tuple[str, str], str]) -> str:
    return rx_dict.get((row['timestamp'], row['serial_port']))

In [79]:
def get_timestamp_rx(path: str) -> str:
    """Returns the timestamp of the RX command respective of the TX command of the row of by iterating every line 
    in the log and checking if "TX" or "OPTICALTX" is in line and if it equals to the "tx_data" of the row and if the timestamps 
    match, if it evaluates to true it searches for a "RX response" with the same "serial_port" of the "TX command".

    Args:
        row (pd.Series): the row of the pd.DataFrame.
        path (str): Logs path.

    Returns:
        str: timestamp of the row's tx command.
    """

    timestamps = defaultdict(str)
    for equipment in logs['equipment_type'].unique():
        for run_id in logs['run_id'].unique():
            for test_case in logs['test_case'].unique():
                file = open_file(path, equipment, run_id, test_case)
                log_lines = file.readlines()
                for i, line in enumerate(log_lines):
                    if 'OPTICALTX:' in line or 'TX:' in line:
                        try:
                            j = 1
                            while True:
                                rx_line = log_lines[i+j]
                                if 'OPTICALRX:' in rx_line:
                                    timestamps[(line[1:24], line[40:49])] = rx_line[1:24]
                                    break
                                elif 'OPTICALTX:' in rx_line:
                                    j += 1
                                    continue
                                elif 'RX:' in rx_line:
                                    timestamps[(line[1:24], line[40:45])] = rx_line[1:24]
                                    break
                                elif 'TX:' in rx_line:
                                    j += 1
                                    continue
                                else:
                                    print('não encontrado', test_case)
                        except IndexError:
                            pass
                file.close()
    return timestamps

def apply_timestamp(row: pd.Series, timestamps_rx_dict: Dict[Tuple[str, str], str]) -> str:
    return timestamps_rx_dict.get((row['timestamp'], row['serial_port']))

In [80]:
def get_response_time(row) -> pd.Timestamp:
    """Returns the response time by subtracting the "timestamp_rx" and "timestamp" of the row.

    Args:
        row (pd.Series): the row of the pd.DataFrame.

    Returns:
        pd.Timestamp: response time.
    """

    if row['timestamp'] < row['timestamp_rx']:
        response_time = row['timestamp_rx'] - row['timestamp']
        return response_time
    else:
        response_time = row['timestamp'] - row['timestamp_rx']
        return response_time

In [81]:
def microseconds_to_seconds(row) -> pd.Timestamp:
    """Returns the conversion of the response time to microseconds.

    Args:
        row (pd.Series): the row of the pd.DataFrame.

    Returns:
        pd.Timestamp: response time in microseconds.
    """

    response_time = (row['response_time'].microseconds) * 10e-4
    return response_time

In [99]:
def get_command_protocol(row: pd.Series) -> str:
    """Returns the command protocol, if there is one, by checking the "tx_data" of the row.

    Args:
        row (pd.Series): the row of the pd.DataFrame.

    Returns:
        str: command protocol.
    """
    try:
        if row['tx_data'][:2] == '02' and row['tx_data'][-1:-3:-1] == '30':
            return 'Production'
        elif len(row['tx_data'].replace(' ', ''))/2 == 66 or len(row['rx_data'].replace(' ', ''))/2 == 258:
            return 'ABNT'
        else:
            return 'protocol not identified'
    except (TypeError, AttributeError):
        return 'None'

In [100]:
def get_command_name(row):
    """Returns the command name, if there is one, by checking the "tx_data" of the row.

    Args:
        row (pd.Series): the row of the pd.DataFrame.

    Returns:
        _type_: command name
    """

    try:    
        if row['command_protocol'] == 'Production':
            command = row['tx_data'][3:5].lower()
            protocol_command = commands_dictionary.production_commands.get(command)
            name = protocol_command.get('name')
            return name
        elif row['command_protocol'] == 'ABNT':
            command = row['tx_data'][:2].lower()
            protocol_command = commands_dictionary.abnt_commands.get(command)
            if protocol_command is None:
                protocol_command = commands_dictionary.abnt_extended_commands.get(command)
                name = protocol_command.get('name')
                return name
            else:
                return protocol_command.get('name')
    except (TypeError, AttributeError):
        return 'name not identified'

# Aplicação das funções

In [84]:
logs['equipment_type'] = get_equipment_type(path_logs)

In [85]:
logs['run_id'] = logs.apply(get_run_ids, path=path_logs, axis=1)
logs = logs.explode('run_id', ignore_index=True)

In [86]:
logs['test_case'] = logs.apply(get_test_case, path=path_logs, axis=1)
logs = logs.explode('test_case', ignore_index=True)
logs

,equipment_type,run_id,test_case
0,8721,220818_171756_851_8721_win,10000
1,8721,220818_171756_851_8721_win,10001
2,8721,220818_171756_851_8721_win,10002
3,8721,220818_171756_851_8721_win,10005
4,8721,220818_171756_851_8721_win,10006
...,...,...,...
1697,8721,220905_220107_851_8721_linux,9995
1698,8721,220905_220107_851_8721_linux,9996
1699,8721,220905_220107_851_8721_linux,9997
1700,8721,220905_220107_851_8721_linux,9998


In [87]:
logs['timestamp'] = logs.apply(get_timestamp, path=path_logs, axis=1)
logs = logs.explode('timestamp', ignore_index=True)
logs

,equipment_type,run_id,test_case,timestamp
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676
...,...,...,...,...
162128,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:24.675
162129,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.605
162130,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.999
162131,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:26.916


In [88]:
bauds = get_baud_rate(path_logs)
logs['baud_rate'] = logs.apply(apply_baud_rate, baud_dict=bauds, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535,9600
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570,9600
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729,9600
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301,9600
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676,9600
...,...,...,...,...,...
162128,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:24.675,9600
162129,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.605,9600
162130,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.999,9600
162131,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:26.916,9600


In [89]:
# same_test_case = False
# logs['baud_rate'], same_test_case = logs.apply(get_baud_rate, path=path_logs, same_test_case=same_test_case, axis=1)
# logs

In [90]:
slots = get_slot(path_logs)
logs['slot'] = logs.apply(apply_slot, slots_dict=slots, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535,9600,B
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570,9600,B
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729,9600,B
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301,9600,OPT
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676,9600,OPT
...,...,...,...,...,...,...
162128,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:24.675,9600,OPT
162129,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.605,9600,OPT
162130,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.999,9600,OPT
162131,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:26.916,9600,OPT


In [91]:
serial_ports = get_serial_port(path_logs)
logs['serial_port'] = logs.apply(apply_serial_port, serial_ports_dict=serial_ports, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535,9600,B,COM22
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570,9600,B,COM22
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729,9600,B,COM22
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301,9600,OPT,COM20
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676,9600,OPT,COM20
...,...,...,...,...,...,...,...
162128,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:24.675,9600,OPT,OPTICALTX
162129,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.605,9600,OPT,OPTICALTX
162130,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.999,9600,OPT,OPTICALTX
162131,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:26.916,9600,OPT,OPTICALTX


In [92]:
tx_datas = get_tx_data(path_logs)
logs['tx_data'] = logs.apply(apply_tx_data, tx_dict=tx_datas, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535,9600,B,COM22,02 01 07 00 09 0C 03
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570,9600,B,COM22,02 01 07 00 00 05 03
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729,9600,B,COM22,02 01 07 01 08 0C 03
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301,9600,OPT,COM20,E2 12 34 56 2F 01 03 01 00 00 00 00 00 00 00 0...
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676,9600,OPT,COM20,E2 12 34 56 2F 01 04 01 00 00 00 00 00 00 00 0...
...,...,...,...,...,...,...,...,...
162128,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:24.675,9600,OPT,OPTICALTX,20 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...
162129,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.605,9600,OPT,OPTICALTX,80 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...
162130,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.999,9600,OPT,OPTICALTX,21 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...
162131,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:26.916,9600,OPT,OPTICALTX,29 12 34 56 07 11 0A 06 00 00 00 00 00 00 00 0...


In [93]:
rx_datas = get_rx_data(path_logs)
logs['rx_data'] = logs.apply(apply_rx_data, rx_dict=rx_datas, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 80 00 00 00 00 00 00 00 00 00 0...
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 80 00 00 00 00 00 0...
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 02 00 FA 3F 01 0...
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301,9600,OPT,COM20,E2 12 34 56 2F 01 03 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 00 00 00 00 00 0...
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676,9600,OPT,COM20,E2 12 34 56 2F 01 04 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 01 00 00 00 00 0...
...,...,...,...,...,...,...,...,...,...
162128,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:24.675,9600,OPT,OPTICALTX,20 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,20 12 34 56 78 14 52 09 07 11 08 06 00 00 20 0...
162129,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.605,9600,OPT,OPTICALTX,80 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,80 12 34 56 78 02 02 02 02 02 02 00 00 19 19 1...
162130,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.999,9600,OPT,OPTICALTX,21 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,21 12 34 56 78 14 52 10 07 11 08 06 00 00 20 0...
162131,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:26.916,9600,OPT,OPTICALTX,29 12 34 56 07 11 0A 06 00 00 00 00 00 00 00 0...,40 12 34 56 78 02 00 01 00 00 00 00 00 00 00 0...


In [94]:
timestamps = get_timestamp_rx(path_logs)
logs['timestamp_rx'] = logs.apply(apply_timestamp, timestamps_rx_dict=timestamps, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,timestamp_rx
0,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.535,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 80 00 00 00 00 00 00 00 00 00 0...,19/08/2022 04:53:39.564
1,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.570,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 80 00 00 00 00 00 0...,19/08/2022 04:53:39.724
2,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:39.729,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 02 00 FA 3F 01 0...,19/08/2022 04:53:39.979
3,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.301,9600,OPT,COM20,E2 12 34 56 2F 01 03 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 00 00 00 00 00 0...,19/08/2022 04:53:40.669
4,8721,220818_171756_851_8721_win,10000,19/08/2022 04:53:40.676,9600,OPT,COM20,E2 12 34 56 2F 01 04 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 01 00 00 00 00 0...,19/08/2022 04:53:41.35
...,...,...,...,...,...,...,...,...,...,...
162128,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:24.675,9600,OPT,OPTICALTX,20 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,20 12 34 56 78 14 52 09 07 11 08 06 00 00 20 0...,05/09/2022 23:14:25.596
162129,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.605,9600,OPT,OPTICALTX,80 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,80 12 34 56 78 02 02 02 02 02 02 00 00 19 19 1...,05/09/2022 23:14:25.964
162130,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:25.999,9600,OPT,OPTICALTX,21 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,21 12 34 56 78 14 52 10 07 11 08 06 00 00 20 0...,05/09/2022 23:14:26.364
162131,8721,220905_220107_851_8721_linux,9999,05/09/2022 23:14:26.916,9600,OPT,OPTICALTX,29 12 34 56 07 11 0A 06 00 00 00 00 00 00 00 0...,40 12 34 56 78 02 00 01 00 00 00 00 00 00 00 0...,05/09/2022 23:14:27.266


In [95]:
logs['timestamp'] = pd.to_datetime(logs['timestamp'], dayfirst=True)
logs['timestamp_rx'] = pd.to_datetime(logs['timestamp_rx'], dayfirst=True)
logs['response_time'] = logs.apply(get_response_time, axis=1)

In [96]:
logs['response_time'] = logs.apply(microseconds_to_seconds, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,timestamp_rx,response_time
0,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:39.535,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 80 00 00 00 00 00 00 00 00 00 0...,2022-08-19 04:53:39.564,29.0
1,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:39.570,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 80 00 00 00 00 00 0...,2022-08-19 04:53:39.724,154.0
2,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:39.729,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 02 00 FA 3F 01 0...,2022-08-19 04:53:39.979,250.0
3,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:40.301,9600,OPT,COM20,E2 12 34 56 2F 01 03 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 00 00 00 00 00 0...,2022-08-19 04:53:40.669,368.0
4,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:40.676,9600,OPT,COM20,E2 12 34 56 2F 01 04 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 01 00 00 00 00 0...,2022-08-19 04:53:41.350,674.0
...,...,...,...,...,...,...,...,...,...,...,...
162128,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:24.675,9600,OPT,OPTICALTX,20 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,20 12 34 56 78 14 52 09 07 11 08 06 00 00 20 0...,2022-09-05 23:14:25.596,921.0
162129,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:25.605,9600,OPT,OPTICALTX,80 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,80 12 34 56 78 02 02 02 02 02 02 00 00 19 19 1...,2022-09-05 23:14:25.964,359.0
162130,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:25.999,9600,OPT,OPTICALTX,21 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,21 12 34 56 78 14 52 10 07 11 08 06 00 00 20 0...,2022-09-05 23:14:26.364,365.0
162131,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:26.916,9600,OPT,OPTICALTX,29 12 34 56 07 11 0A 06 00 00 00 00 00 00 00 0...,40 12 34 56 78 02 00 01 00 00 00 00 00 00 00 0...,2022-09-05 23:14:27.266,350.0


In [97]:
logs.iloc[923]

equipment_type                          8721
run_id            220818_171756_851_8721_win
test_case                              10082
timestamp                                NaT
baud_rate                               None
slot                                    None
serial_port                             None
tx_data                                 None
rx_data                                 None
timestamp_rx                             NaT
response_time                            NaN
Name: 923, dtype: object

In [101]:
logs['command_protocol'] = logs.apply(get_command_protocol, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,timestamp_rx,response_time,command_protocol
0,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:39.535,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 80 00 00 00 00 00 00 00 00 00 0...,2022-08-19 04:53:39.564,29.0,Production
1,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:39.570,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 80 00 00 00 00 00 0...,2022-08-19 04:53:39.724,154.0,Production
2,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:39.729,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 02 00 FA 3F 01 0...,2022-08-19 04:53:39.979,250.0,Production
3,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:40.301,9600,OPT,COM20,E2 12 34 56 2F 01 03 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 00 00 00 00 00 0...,2022-08-19 04:53:40.669,368.0,ABNT
4,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:40.676,9600,OPT,COM20,E2 12 34 56 2F 01 04 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 01 00 00 00 00 0...,2022-08-19 04:53:41.350,674.0,ABNT
...,...,...,...,...,...,...,...,...,...,...,...,...
162128,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:24.675,9600,OPT,OPTICALTX,20 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,20 12 34 56 78 14 52 09 07 11 08 06 00 00 20 0...,2022-09-05 23:14:25.596,921.0,ABNT
162129,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:25.605,9600,OPT,OPTICALTX,80 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,80 12 34 56 78 02 02 02 02 02 02 00 00 19 19 1...,2022-09-05 23:14:25.964,359.0,ABNT
162130,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:25.999,9600,OPT,OPTICALTX,21 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,21 12 34 56 78 14 52 10 07 11 08 06 00 00 20 0...,2022-09-05 23:14:26.364,365.0,ABNT
162131,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:26.916,9600,OPT,OPTICALTX,29 12 34 56 07 11 0A 06 00 00 00 00 00 00 00 0...,40 12 34 56 78 02 00 01 00 00 00 00 00 00 00 0...,2022-09-05 23:14:27.266,350.0,ABNT


In [102]:
logs['command_name'] = logs.apply(get_command_name, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,timestamp_rx,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:39.535,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 80 00 00 00 00 00 00 00 00 00 0...,2022-08-19 04:53:39.564,29.0,Production,CALIB_CMD_READ_DATA
1,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:39.570,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 80 00 00 00 00 00 0...,2022-08-19 04:53:39.724,154.0,Production,CALIB_CMD_READ_DATA
2,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:39.729,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 02 00 FA 3F 01 0...,2022-08-19 04:53:39.979,250.0,Production,CALIB_CMD_READ_DATA
3,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:40.301,9600,OPT,COM20,E2 12 34 56 2F 01 03 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 00 00 00 00 00 0...,2022-08-19 04:53:40.669,368.0,ABNT,ABNT_CMD_E2
4,8721,220818_171756_851_8721_win,10000,2022-08-19 04:53:40.676,9600,OPT,COM20,E2 12 34 56 2F 01 04 01 00 00 00 00 00 00 00 0...,E2 12 34 56 78 2F 01 01 01 01 01 00 00 00 00 0...,2022-08-19 04:53:41.350,674.0,ABNT,ABNT_CMD_E2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162128,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:24.675,9600,OPT,OPTICALTX,20 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,20 12 34 56 78 14 52 09 07 11 08 06 00 00 20 0...,2022-09-05 23:14:25.596,921.0,ABNT,ABNT_CMD_20
162129,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:25.605,9600,OPT,OPTICALTX,80 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,80 12 34 56 78 02 02 02 02 02 02 00 00 19 19 1...,2022-09-05 23:14:25.964,359.0,ABNT,ABNT_CMD_80
162130,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:25.999,9600,OPT,OPTICALTX,21 12 34 56 00 00 00 00 00 00 00 00 00 00 00 0...,21 12 34 56 78 14 52 10 07 11 08 06 00 00 20 0...,2022-09-05 23:14:26.364,365.0,ABNT,ABNT_CMD_21
162131,8721,220905_220107_851_8721_linux,9999,2022-09-05 23:14:26.916,9600,OPT,OPTICALTX,29 12 34 56 07 11 0A 06 00 00 00 00 00 00 00 0...,40 12 34 56 78 02 00 01 00 00 00 00 00 00 00 0...,2022-09-05 23:14:27.266,350.0,ABNT,ABNT_CMD_29
